## 011 データを読み込む

In [222]:
import pandas as pd
import re
import seaborn as sns

In [223]:
uriage_data = pd.read_csv('./input/100knoks/01-02/uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [224]:
uriage_data.describe()

,item_price
count,2612.000000
mean,1296.401225
std,717.955460
min,100.000000
25%,700.000000
50%,1300.000000
75%,1900.000000
max,2600.000000


In [225]:
uriage_data.dtypes

purchase_date     object
item_name         object
item_price       float64
customer_name     object
dtype: object

In [226]:
#kokyaku_data = pd.read_excel('./input/100knoks/01-02/kokyaku_daicho.xlsx')
kokyaku_data = pd.read_csv('./input/100knoks/01-02/kokyaku_daicho.csv')
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,2017年2月16日
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,2017年5月17日
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018年1月27日


## 012 データの揺れを見る

In [227]:
uriage_data['purchase_date'].head()

0    2019-06-13 18:02:34
1    2019-07-13 13:05:29
2    2019-05-11 19:42:07
3    2019-02-12 23:40:45
4    2019-04-22 03:09:35
Name: purchase_date, dtype: object

In [228]:
uriage_data['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [229]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

## 013 揺れがあるまま集計

In [230]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime("%Y%m")
uriage_data['purchase_month'].head()

0    201906
1    201907
2    201905
3    201902
4    201904
Name: purchase_month, dtype: object

In [231]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [232]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='sum', fill_value=0)
res

item_price                                                  \
item_name             商品W  商 品 n  商品E   商品M   商品P   商品S   商品W   商品X 商  品O   
purchase_month                                                              
201901                  0   1400    0     0     0     0     0     0     0   
201902                  0      0    0     0     0     0     0  2400     0   
201903                  0      0  500  1300  1600     0     0     0     0   
201904               2300      0    0     0     0     0     0     0     0   
201905                  0      0    0     0     0  1900     0     0     0   
201906                  0      0    0     0     0     0  2300     0     0   
201907                  0      0    0     0     0     0     0     0     0   

                      ...                                                  \
item_name      商  品Q  ...   商品k   商品l   商品o   商品p   商品r   商品s   商品t   商品v   
purchase_month        ...                                                   
201901             0  ...  1100  1200  1500     0     0     0     0     0   
201902             0  ...     0     0     0     0     0  1900  2000  2200   
201903             0  ...     0     0     0     0     0     0     0     0   
201904          1700  ...     0     0     0     0     0  1900     0     0   
201905             0  ...     0  1200     0     0     0     0     0     0   
201906             0  ...     0     0     0  1600     0     0     0     0   
201907             0  ...     0     0  1500     0  1800     0     0     0   

                            
item_name        商品x   商品y  
purchase_month              
201901             0     0  
201902             0     0  
201903             0     0  
201904             0     0  
201905             0  2500  
201906          2400     0  
201907             0     0  

[7 rows x 99 columns]

## 014 商品名の揺れを補正

無駄なスペースを削除して、英数字を半角全角、大文字小文字を統一する

In [233]:
print(len(pd.unique(uriage_data['item_name'])))

99


In [234]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
print(len(pd.unique(uriage_data['item_name'])))

78


In [235]:
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', '')
print(len(pd.unique(uriage_data['item_name'])))

78


In [236]:
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
print(len(pd.unique(uriage_data['item_name'])))

26


In [237]:
uriage_data.sort_values(by='item_name', ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,201905


In [238]:
print(uriage_data['item_name'].unique())
print(len(uriage_data['item_name'].unique()))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


## 015 金額の欠損を補完する

In [239]:
uriage_data.isnull().sum()

purchase_date       0
item_name           0
item_price        387
customer_name       0
purchase_month      0
dtype: int64

In [240]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [241]:
flg_is_null = uriage_data['item_price'].isnull()
flg_is_null

0       False
1        True
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996     True
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [242]:
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price

uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


### locメソッドの使い方
locメソッドは、条件を指定してその条件に合致するデータフレームを抽出することが可能。
１つ目に条件を渡す。今回の場合は金額がNaNのデータなので、flag_is_nullを渡す。2番目に取り出すデータのカラム名を渡すので
今回は'item_name'を渡している。

In [243]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [244]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg, '最大値', uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].max(skipna=False))
    print(trg, '最小値', uriage_data.loc[uriage_data['item_name'] == trg, 'item_price'].min(skipna=False))
    print('-'*80)

商品A 最大値 100.0
商品A 最小値 100.0
--------------------------------------------------------------------------------
商品B 最大値 200.0
商品B 最小値 200.0
--------------------------------------------------------------------------------
商品C 最大値 300.0
商品C 最小値 300.0
--------------------------------------------------------------------------------
商品D 最大値 400.0
商品D 最小値 400.0
--------------------------------------------------------------------------------
商品E 最大値 500.0
商品E 最小値 500.0
--------------------------------------------------------------------------------
商品F 最大値 600.0
商品F 最小値 600.0
--------------------------------------------------------------------------------
商品G 最大値 700.0
商品G 最小値 700.0
--------------------------------------------------------------------------------
商品H 最大値 800.0
商品H 最小値 800.0
--------------------------------------------------------------------------------
商品I 最大値 900.0
商品I 最小値 900.0
--------------------------------------------------------------------------------
商品J 最大値 1000.0
商品J 

## 顧客名の揺れを補正する

In [245]:
kokyaku_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [246]:
uriage_data['customer_name'].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [247]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '')
kokyaku_data.head()


,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017年2月16日
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017年5月17日
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018年1月27日


## 日付の揺れを補正する

In [248]:
kokyaku_data['登録日'].astype('str').str.isdigit().sum()

0

In [249]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 登録日, Length: 200, dtype: bool

In [250]:
flg_is_jstring = kokyaku_data['登録日'].astype('str').str.find('年') > 0
flg_is_jstring

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 登録日, Length: 200, dtype: bool

In [251]:
flg_is_string = kokyaku_data['登録日'].astype('str').str.find('/') > 0
flg_is_string

0       True
1      False
2       True
3      False
4      False
       ...  
195     True
196     True
197     True
198     True
199     True
Name: 登録日, Length: 200, dtype: bool

In [252]:
from_serial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
from_serial

Series([], Name: 登録日, dtype: datetime64[ns])

In [253]:
from_jstring = pd.to_datetime(kokyaku_data.loc[flg_is_jstring, '登録日'], format="%Y年%m月%d日")

In [254]:
from_string = pd.to_datetime(kokyaku_data.loc[flg_is_string, '登録日'], format="%Y/%m/%d")
from_string

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [255]:
kokyaku_data['登録日'] = pd.concat([from_jstring, from_string])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [256]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime("%Y%m")
res = kokyaku_data.groupby('登録年月').count()['顧客名']
print(res)
print(res.sum())

登録年月
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


## 顧客名をキーにデータを結合 Join

In [261]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
join_data.drop('customer_name', axis=1)
join_data

,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,尾形小雁,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13,201807
